### 实践：手写数字识别任务
- [官网地址](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/02_paddle2.0_develop/01_quick_start_cn.html)

In [1]:
import paddle
import warnings
warnings.filterwarnings("ignore")

In [3]:
paddle.__version__

'2.2.1'

#### 数据集的加载与预处理

>[paddle.vision](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/vision/Overview_cn.html)目录是 paddle 在视觉领域的最高层API
>
>transform.Normalize 图像归一化处理
>
>1、用统一的均值和标准差对图像进行归一化处理
>
>2、对每个通道进行不同的均值和标准差进行归一化处理
>
><font color="red">CHW：一种数据格式，不知道真正的意思？？？</font>

In [2]:
import paddle.vision.transforms as T

In [4]:
transform = T.Normalize(mean=[127.5],std=[127.5],data_format="CHW")

In [5]:
transform

In [7]:
# 下载数据集
train_dataset = paddle.vision.MNIST(mode="train",transform=transform)
val_dataset = paddle.vision.MNIST(mode="test",transform=transform)

#### 设计模型

>[paddle.nn](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Overview_cn.html) 目录下包含飞浆框架支持的神经网络层和相关的API
>
>Sequential 顺序容器，数据按照子层的顺序计算。**将上一层的输出结果，当作下一层的输入结果【自己的理解】**
>
>Flatten 展平层，将一个连续维度的 Tensor 展平为一维的 Tensor
>
>Linear 线性变化层，out=XW+b，其中W和b是需要学习的，784是上一层输入的单元数目，512是该层输出的单元数目
>
>ReLU 激活层，ReLU(x)=max(0,x) x为输入的 tensor
>
>Dropout 正则化层，按照给定的概率随机的将一些神经元的输出设置为0，目的是为了防止神经元节点之间过拟合

In [9]:
mnist = paddle.nn.Sequential(
    paddle.nn.Flatten(),
    paddle.nn.Linear(784,512),
    paddle.nn.ReLU(),
    paddle.nn.Dropout(0.2),
    paddle.nn.Linear(512,10)
)

#### 模型训练


>[paddle.Model](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/Model_cn.html) 一个具备模型训练、测试、推理的神经网络对象，方便模型的所有操作
>
>Model.prepare 配置模型所需的部件，比如：优化器、损失函数和评价指标
>
>paddle.optimizer.Adam 优化器，**利用梯度的一阶矩估计和二阶矩估计动态调整每个参数的学习率**
>
>paddle.nn.CrossEntropLoss  损失函数，计算输入input和标签label的交叉熵损失，可用于训练一个 n 类分类器
>
>paddle.metric.Accuracy 评估指标，计算准确率

In [10]:
model = paddle.Model(mnist)

In [11]:
model.prepare(
    paddle.optimizer.Adam(parameters=model.parameters()),
    paddle.nn.CrossEntropyLoss(),
    paddle.metric.Accuracy()
)

In [13]:
model.fit(
    train_dataset,
    epochs=5,
    batch_size=64,
    verbose=1
)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 938/938 [==============================] - loss: 0.2681 - acc: 0.9011 - 17ms/step          
Epoch 2/5
step 938/938 [==============================] - loss: 0.2362 - acc: 0.9494 - 22ms/step          
Epoch 3/5
step 938/938 [==============================] - loss: 0.0444 - acc: 0.9595 - 22ms/step          
Epoch 4/5
step 938/938 [==============================] - loss: 0.0198 - acc: 0.9638 - 23ms/step          
Epoch 5/5
step 938/938 [==============================] - loss: 0.1433 - acc: 0.9672 - 22ms/step          


#### 模型评估

>evaluate 在测试数据上评估模型的损失函数和评估指标
>
>**verbose=0，不打印日志；verbose=1，进度条打印日志；verbose=2，一行一行打印日志；**

In [14]:
model.evaluate(val_dataset,verbose=0)

{'loss': [2.264979e-06], 'acc': 0.9724}

In [16]:
model.summary()

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Flatten-2     [[64, 1, 28, 28]]        [64, 784]              0       
   Linear-2         [[64, 784]]           [64, 512]           401,920    
    ReLU-1          [[64, 512]]           [64, 512]              0       
   Dropout-1        [[64, 512]]           [64, 512]              0       
   Linear-3         [[64, 512]]            [64, 10]            5,130     
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
---------------------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 1.14
Params size (MB): 1.55
Estimated Total Size (MB): 2.88
---------------------------------------------------------------------------



{'total_params': 407050, 'trainable_params': 407050}